In [ ]:
import pandas as pd

In [ ]:
from sklearn.impute import SimpleImputer
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Loading the dataset given
df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/players_22.csv") #player_22
df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/male_players (legacy).csv") #male_legacy

<ipython-input-62-f5945c0551cb>:2: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/players_22.csv") #player_22
<ipython-input-62-f5945c0551cb>:3: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/male_players (legacy).csv") #male_legacy


In [ ]:
df.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [ ]:
df.dtypes

player_id            int64
player_url          object
fifa_version         int64
fifa_update          int64
fifa_update_date    object
                     ...  
cb                  object
rcb                 object
rb                  object
gk                  object
player_face_url     object
Length: 110, dtype: object

In [ ]:
#drop things that be useless
df.columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential',
       ...
       'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk',
       'player_face_url'],
      dtype='object', length=110)

In [ ]:
df.drop(columns = ['player_id', 'player_url', 'fifa_version', 'fifa_update', 'fifa_update_date', 'player_face_url', 'short_name', 'long_name'], inplace = True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 102 entries, player_positions to gk
dtypes: float64(18), int64(42), object(42)
memory usage: 125.7+ MB


In [ ]:
y = df['overall'] #our target variable

In [ ]:
quant = df.select_dtypes(include = [np.number]) #we get all the quantitative features

In [ ]:
corr_matrix = quant.corr()

In [ ]:
corr_matrix['overall'].sort_values(ascending = False)

overall                           1.000000
movement_reactions                0.845753
potential                         0.695362
mentality_composure               0.683307
passing                           0.681114
wage_eur                          0.611368
dribbling                         0.606575
release_clause_eur                0.578914
value_eur                         0.566827
physic                            0.513279
attacking_short_passing           0.499757
mentality_vision                  0.493282
international_reputation          0.485234
skill_long_passing                0.483808
shooting                          0.482498
power_shot_power                  0.480297
age                               0.458390
skill_ball_control                0.457011
skill_curve                       0.415863
power_long_shots                  0.409086
mentality_aggression              0.398310
attacking_crossing                0.396833
skill_fk_accuracy                 0.387269
goalkeeping

In [ ]:
#we are dropping columns that have a correlation less than 0.4 and greater than -0.4
for column in quant.columns:
  if corr_matrix['overall'][column] < 0.4 and corr_matrix['overall'][column] > -0.4:
    quant.drop(column, axis = 1, inplace = True)

In [ ]:
quant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   overall                   161583 non-null  int64  
 1   potential                 161583 non-null  int64  
 2   value_eur                 159530 non-null  float64
 3   wage_eur                  159822 non-null  float64
 4   age                       161583 non-null  int64  
 5   international_reputation  161583 non-null  int64  
 6   release_clause_eur        103622 non-null  float64
 7   shooting                  143614 non-null  float64
 8   passing                   143614 non-null  float64
 9   dribbling                 143614 non-null  float64
 10  physic                    143614 non-null  float64
 11  attacking_short_passing   161583 non-null  int64  
 12  skill_curve               161583 non-null  int64  
 13  skill_long_passing        161583 non-null  i

In [ ]:
corr_matrix = quant.corr()

In [ ]:
corr_matrix['overall'].sort_values(ascending = False)

overall                     1.000000
movement_reactions          0.845753
potential                   0.695362
mentality_composure         0.683307
passing                     0.681114
wage_eur                    0.611368
dribbling                   0.606575
release_clause_eur          0.578914
value_eur                   0.566827
physic                      0.513279
attacking_short_passing     0.499757
mentality_vision            0.493282
international_reputation    0.485234
skill_long_passing          0.483808
shooting                    0.482498
power_shot_power            0.480297
age                         0.458390
skill_ball_control          0.457011
skill_curve                 0.415863
power_long_shots            0.409086
Name: overall, dtype: float64

In [ ]:
#we need to impute cause there are missing values
imputer = SimpleImputer(strategy = 'median')
q_columns = quant.columns
quant = imputer.fit_transform(quant)
quant = pd.DataFrame(quant, columns = q_columns)

In [ ]:
quant

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,93.0,95.0,100500000.0,550000.0,27.0,5.0,1300000.0,89.0,86.0,96.0,63.0,89.0,89.0,76.0,96.0,94.0,80.0,88.0,90.0,59.0
1,92.0,92.0,79000000.0,375000.0,29.0,5.0,1300000.0,93.0,81.0,91.0,79.0,82.0,88.0,72.0,92.0,90.0,94.0,93.0,81.0,59.0
2,90.0,90.0,54500000.0,275000.0,30.0,5.0,1300000.0,86.0,83.0,92.0,64.0,86.0,85.0,76.0,90.0,89.0,86.0,90.0,84.0,59.0
3,90.0,90.0,52500000.0,275000.0,32.0,5.0,1300000.0,91.0,81.0,86.0,86.0,84.0,80.0,76.0,90.0,85.0,93.0,88.0,83.0,59.0
4,90.0,90.0,63500000.0,300000.0,28.0,5.0,1300000.0,54.0,58.0,63.0,66.0,42.0,25.0,41.0,31.0,89.0,42.0,25.0,20.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,1.0,239000.0,48.0,40.0,47.0,44.0,38.0,39.0,33.0,45.0,39.0,64.0,31.0,42.0,40.0
161579,46.0,58.0,110000.0,750.0,19.0,1.0,193000.0,25.0,29.0,34.0,57.0,30.0,24.0,25.0,32.0,42.0,33.0,25.0,31.0,35.0
161580,46.0,58.0,110000.0,500.0,19.0,1.0,193000.0,36.0,43.0,46.0,53.0,51.0,30.0,43.0,35.0,50.0,51.0,32.0,40.0,35.0
161581,46.0,70.0,150000.0,500.0,17.0,1.0,383000.0,50.0,36.0,46.0,42.0,40.0,38.0,26.0,43.0,45.0,42.0,47.0,44.0,43.0


In [ ]:
#after imputing then we need to scale
scaler = StandardScaler()
quant = pd.DataFrame(scaler.fit_transform(quant), columns = quant.columns)
quant

,overall,potential,value_eur,wage_eur,age,international_reputation,release_clause_eur,shooting,passing,dribbling,physic,attacking_short_passing,skill_curve,skill_long_passing,skill_ball_control,movement_reactions,power_shot_power,power_long_shots,mentality_vision,mentality_composure
0,3.877514,3.875315,16.447428,24.697104,0.401872,10.148137,-0.224202,2.783978,2.926570,3.480487,-0.211839,2.086204,2.306588,1.536683,2.281310,3.535775,1.499832,2.148684,2.588430,0.085565
1,3.735485,3.396013,12.846177,16.681825,0.830120,10.148137,-0.224202,3.088467,2.421008,2.965866,1.529711,1.609554,2.251267,1.273977,2.039859,3.099571,2.401757,2.411698,1.957672,0.085565
2,3.451428,3.076478,8.742427,12.101666,1.044244,10.148137,-0.224202,2.555612,2.623233,3.068791,-0.102992,1.881925,2.085305,1.536683,1.919133,2.990520,1.886371,2.253890,2.167924,0.085565
3,3.451428,3.076478,8.407427,12.101666,1.472491,10.148137,-0.224202,2.936223,2.421008,2.451245,2.291639,1.745740,1.808701,1.536683,1.919133,2.554317,2.337334,2.148684,2.097840,0.085565
4,3.451428,3.076478,10.249927,13.246706,0.615996,10.148137,-0.224202,0.119701,0.095424,0.083989,0.114701,-1.114157,-1.233943,-0.761996,-1.642271,2.990520,-0.948252,-1.165296,-2.317466,0.085565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,-2.797832,-1.556774,-0.367900,-0.461712,-1.525244,-0.279299,-0.330265,-0.337032,-1.724598,-1.562798,-2.279930,-1.386528,-0.459452,-1.287408,-0.797192,-2.462022,0.469059,-0.849678,-0.775613,-1.639346
161579,-2.797832,-2.036076,-0.367900,-0.459422,-1.311120,-0.279299,-0.334863,-2.087843,-2.836834,-2.900813,-0.864920,-1.931270,-1.289264,-1.812820,-1.581909,-2.134869,-1.528062,-1.165296,-1.546539,-2.093270
161580,-2.797832,-2.036076,-0.367900,-0.470872,-1.311120,-0.279299,-0.334863,-1.250499,-1.421261,-1.665723,-1.300308,-0.501322,-0.957339,-0.630643,-1.400820,-1.262462,-0.368443,-0.797076,-0.915781,-2.093270
161581,-2.797832,-0.118868,-0.361200,-0.470872,-1.739368,-0.279299,-0.315870,-0.184788,-2.129048,-1.665723,-2.497623,-1.250342,-0.514773,-1.747144,-0.917918,-1.807717,-0.948252,-0.008033,-0.635445,-1.366992


In [ ]:
quant.drop('overall', axis = 1, inplace = True)

**Done Processing quantitative data!**

**Processing categorical data**

In [ ]:
cat = df.select_dtypes(include = ['object'])
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 42 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   player_positions  161583 non-null  object
 1   dob               161583 non-null  object
 2   league_name       159810 non-null  object
 3   club_name         159810 non-null  object
 4   club_position     159810 non-null  object
 5   club_loaned_from  9538 non-null    object
 6   club_joined_date  150272 non-null  object
 7   nationality_name  161583 non-null  object
 8   nation_position   9316 non-null    object
 9   preferred_foot    161583 non-null  object
 10  work_rate         161583 non-null  object
 11  body_type         161583 non-null  object
 12  real_face         161583 non-null  object
 13  player_tags       12663 non-null   object
 14  player_traits     73150 non-null   object
 15  ls                161583 non-null  object
 16  st                161583 non-null  obj

In [ ]:
important_cols = ['preferred_foot', 'body_type']
cat.drop(columns = [column for column in cat.columns if column not in important_cols], inplace =True)

In [ ]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   preferred_foot  161583 non-null  object
 1   body_type       161583 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [ ]:
#one hot encoding
cat = pd.get_dummies(cat).astype(int)
cat

,preferred_foot_Left,preferred_foot_Right,body_type_Lean (170-),body_type_Lean (170-185),body_type_Lean (185+),body_type_Normal (170-),body_type_Normal (170-185),body_type_Normal (185+),body_type_Stocky (170-),body_type_Stocky (170-185),body_type_Stocky (185+),body_type_Unique
0,1,0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,0,0
2,1,0,0,0,0,0,1,0,0,0,0,0
3,0,1,0,0,0,0,0,1,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
161578,0,1,0,1,0,0,0,0,0,0,0,0
161579,1,0,0,0,0,0,0,1,0,0,0,0
161580,0,1,0,0,0,0,1,0,0,0,0,0
161581,0,1,0,0,0,0,1,0,0,0,0,0


**Done Processing Categorical data**

**Bring everything together and train and test the model**

In [ ]:
X = pd.concat([cat, quant], axis = 1)

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
lin_reg = LinearRegression()

In [ ]:
lin_reg.fit(Xtrain, Ytrain) #da training function
lin_reg_pred = lin_reg.predict(Xtest)

In [ ]:
print(f""" Mean Aboslute Error={mean_absolute_error(lin_reg_pred,Ytest)}
          Mean Squared Error = {mean_squared_error(lin_reg_pred,Ytest)}
          Root Mean Squared error = {np.sqrt(mean_squared_error(lin_reg_pred,Ytest))}
          R2 score = {r2_score(lin_reg_pred,Ytest)}
""")

 Mean Aboslute Error=1.5412001828323716
          Mean Squared Error = 3.987445694852016
          Root Mean Squared error = 1.9968589571754978
          R2 score = 0.9123594459472737



In [ ]:
dTree = DecisionTreeRegressor()
dTree.fit(Xtrain, Ytrain)

DecisionTreeRegressor()

In [ ]:
dTree_pred = dTree.predict(Xtest)

In [ ]:
print(f""" Mean Aboslute Error={mean_absolute_error(dTree_pred,Ytest)}
          Mean Squared Error = {mean_squared_error(dTree_pred,Ytest)}
          Root Mean Squared error = {np.sqrt(mean_squared_error(dTree_pred,Ytest))}
          R2 score = {r2_score(dTree_pred,Ytest)}
""")

 Mean Aboslute Error=0.5982609771946653
          Mean Squared Error = 1.4401089210013305
          Root Mean Squared error = 1.2000453828923847
          R2 score = 0.9710728111205364



In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
model = RandomForestRegressor()

In [ ]:
parameter = {
    'n_estimators': [100,200,300],
    'max_depth': [5,7,9]
}

In [ ]:
rand_search = RandomizedSearchCV(model, parameter, scoring = 'r2', cv = 3)

In [ ]:
rand_pred= rand_search.fit(Xtrain, Ytrain)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


In [ ]:
best_params = rand_pred.best_params_
print(f"Best Parameters: {best_params}")

# Best estimator
best_estimator = rand_pred.best_estimator_
print(f"Best Estimator: {best_estimator}")

# Best score
best_score = rand_pred.best_score_
print(f"Best Cross-Validation Score: {best_score}")

Best Parameters: {'n_estimators': 300, 'max_depth': 9}
Best Estimator: RandomForestRegressor(max_depth=9, n_estimators=300)
Best Cross-Validation Score: 0.9741610059944831


In [ ]:
df2.head()

,sofifa_id,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,...,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,...,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,188545,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,...,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,...,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",91,91,129000000.0,270000.0,29,...,50+3,50+3,50+3,62+3,20+3,https://cdn.sofifa.net/players/190/871/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,NaN,https://cdn.sofifa.net/flags/br.png
4,192985,https://sofifa.com/player/192985/kevin-de-bruy...,K. De Bruyne,Kevin De Bruyne,"CM, CAM",91,91,125500000.0,350000.0,30,...,69+3,69+3,69+3,75+3,21+3,https://cdn.sofifa.net/players/192/985/22_120.png,https://cdn.sofifa.net/teams/10/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1325/60.png,https://cdn.sofifa.net/flags/be.png


In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 110 entries, sofifa_id to nation_flag_url
dtypes: float64(16), int64(44), object(50)
memory usage: 16.1+ MB


In [ ]:
df2.dtypes

sofifa_id            int64
player_url          object
short_name          object
long_name           object
player_positions    object
                     ...  
player_face_url     object
club_logo_url       object
club_flag_url       object
nation_logo_url     object
nation_flag_url     object
Length: 110, dtype: object

In [ ]:
df2.columns

Index(['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=101)

In [ ]:
df2.drop(columns = ['sofifa_id', 'player_url', 'club_logo_url', 'club_flag_url', 'player_face_url', 'short_name', 'long_name', 'nation_flag_url', 'nation_logo_url', ], inplace = True)

KeyError: "['sofifa_id', 'player_url', 'club_logo_url', 'club_flag_url', 'player_face_url', 'short_name', 'long_name', 'nation_flag_url', 'nation_logo_url'] not found in axis"

In [ ]:
df2.columns

Index(['player_positions', 'overall', 'potential', 'value_eur', 'wage_eur',
       'age', 'dob', 'height_cm', 'weight_kg', 'club_team_id',
       ...
       'ldm', 'cdm', 'rdm', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 'rb', 'gk'],
      dtype='object', length=101)

In [ ]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Columns: 101 entries, player_positions to gk
dtypes: float64(16), int64(43), object(42)
memory usage: 14.8+ MB


In [ ]:
y = df['overall']

In [ ]:
quant = df.select_dtypes(include = [np.number])

In [ ]:
quant

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
0,158023,15,2,93,95,100500000.0,550000.0,27,169,67,...,NaN,25,21,20,6,11,15,14,8,NaN
1,20801,15,2,92,92,79000000.0,375000.0,29,185,80,...,NaN,22,31,23,7,11,15,14,11,NaN
2,9014,15,2,90,90,54500000.0,275000.0,30,180,80,...,NaN,29,26,26,10,8,11,5,15,NaN
3,41236,15,2,90,90,52500000.0,275000.0,32,195,95,...,NaN,25,41,27,13,15,10,9,12,NaN
4,167495,15,2,90,90,63500000.0,300000.0,28,193,92,...,NaN,25,25,25,87,85,92,90,86,60.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,23,2,46,61,110000.0,700.0,18,180,73,...,40.0,23,21,25,9,13,13,12,7,NaN
161579,269019,23,2,46,58,110000.0,750.0,19,188,83,...,35.0,50,51,45,6,14,8,13,14,NaN
161580,271093,23,2,46,58,110000.0,500.0,19,181,73,...,35.0,36,45,50,8,9,7,14,9,NaN
161581,271555,23,2,46,70,150000.0,500.0,17,175,68,...,43.0,19,17,14,13,12,14,7,13,NaN


In [ ]:
df2.drop(columns = ['mentality_composure','goalkeeping_speed'], inplace = True)

In [ ]:
corr_matrix = quant.corr()

In [ ]:
corr_matrix['overall'].sort_values(ascending = False)

overall                1.000000
movement_reactions     0.845753
potential              0.695362
mentality_composure    0.683307
passing                0.681114
                         ...   
club_jersey_number    -0.187109
league_level          -0.221734
nation_team_id        -0.338501
player_id             -0.368772
fifa_update                 NaN
Name: overall, Length: 63, dtype: float64

In [ ]:
for column in quant.columns:
  if corr_matrix['overall'][column] < 0.6 and corr_matrix['overall'][column] > -0.6:
    quant.drop(column, axis = 1, inplace = True)

In [ ]:
imputer = SimpleImputer(strategy = 'median')
q_columns = quant.columns
quant = imputer.fit_transform(quant)
quant = pd.DataFrame(quant, columns = q_columns)

In [ ]:
quant

,fifa_update,overall,potential,wage_eur,passing,dribbling,movement_reactions,mentality_composure
0,2.0,93.0,95.0,550000.0,86.0,96.0,94.0,59.0
1,2.0,92.0,92.0,375000.0,81.0,91.0,90.0,59.0
2,2.0,90.0,90.0,275000.0,83.0,92.0,89.0,59.0
3,2.0,90.0,90.0,275000.0,81.0,86.0,85.0,59.0
4,2.0,90.0,90.0,300000.0,58.0,63.0,89.0,59.0
...,...,...,...,...,...,...,...,...
161578,2.0,46.0,61.0,700.0,40.0,47.0,39.0,40.0
161579,2.0,46.0,58.0,750.0,29.0,34.0,42.0,35.0
161580,2.0,46.0,58.0,500.0,43.0,46.0,50.0,35.0
161581,2.0,46.0,70.0,500.0,36.0,46.0,45.0,43.0


In [ ]:
scaler = StandardScaler()
quant = pd.DataFrame(scaler.fit_transform(quant), columns = quant.columns)
quant

,fifa_update,overall,potential,wage_eur,passing,dribbling,movement_reactions,mentality_composure
0,0.0,3.877514,3.875315,24.697104,2.926570,3.480487,3.535775,0.085565
1,0.0,3.735485,3.396013,16.681825,2.421008,2.965866,3.099571,0.085565
2,0.0,3.451428,3.076478,12.101666,2.623233,3.068791,2.990520,0.085565
3,0.0,3.451428,3.076478,12.101666,2.421008,2.451245,2.554317,0.085565
4,0.0,3.451428,3.076478,13.246706,0.095424,0.083989,2.990520,0.085565
...,...,...,...,...,...,...,...,...
161578,0.0,-2.797832,-1.556774,-0.461712,-1.724598,-1.562798,-2.462022,-1.639346
161579,0.0,-2.797832,-2.036076,-0.459422,-2.836834,-2.900813,-2.134869,-2.093270
161580,0.0,-2.797832,-2.036076,-0.470872,-1.421261,-1.665723,-1.262462,-2.093270
161581,0.0,-2.797832,-0.118868,-0.470872,-2.129048,-1.665723,-1.807717,-1.366992


In [ ]:
quant.drop('overall', axis = 1, inplace = True)

In [ ]:
cat = df2.select_dtypes(include = ['object'])
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 42 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   player_positions  19239 non-null  object
 1   dob               19239 non-null  object
 2   club_name         19178 non-null  object
 3   league_name       19178 non-null  object
 4   club_position     19178 non-null  object
 5   club_loaned_from  1102 non-null   object
 6   club_joined       18076 non-null  object
 7   nationality_name  19239 non-null  object
 8   nation_position   759 non-null    object
 9   preferred_foot    19239 non-null  object
 10  work_rate         19239 non-null  object
 11  body_type         19239 non-null  object
 12  real_face         19239 non-null  object
 13  player_tags       1441 non-null   object
 14  player_traits     9398 non-null   object
 15  ls                19239 non-null  object
 16  st                19239 non-null  object
 17  rs          

In [ ]:
important_cols = ['preferred_foot', 'body_type','work_rate']
cat.drop(columns = [column for column in cat.columns if column not in important_cols], inplace =True)
important_cols

['preferred_foot', 'body_type', 'work_rate']

In [ ]:
cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   preferred_foot  19239 non-null  object
 1   work_rate       19239 non-null  object
 2   body_type       19239 non-null  object
dtypes: object(3)
memory usage: 451.0+ KB


In [ ]:
#one hot encoding
cat = pd.get_dummies(cat).astype(int)
cat

,preferred_foot_Left,preferred_foot_Right,work_rate_High/High,work_rate_High/Low,work_rate_High/Medium,work_rate_Low/High,work_rate_Low/Low,work_rate_Low/Medium,work_rate_Medium/High,work_rate_Medium/Low,...,body_type_Lean (170-),body_type_Lean (170-185),body_type_Lean (185+),body_type_Normal (170-),body_type_Normal (170-185),body_type_Normal (185+),body_type_Stocky (170-),body_type_Stocky (170-185),body_type_Stocky (185+),body_type_Unique
0,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19234,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
19235,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
19236,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
19237,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [ ]:
X = pd.concat([cat, quant], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

ValueError: Found input variables with inconsistent numbers of samples: [161583, 80]

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

In [ ]:
base_classifier = DecisionTreeClassifier()

In [ ]:
bagging_classifier = BaggingClassifier(base_estimator=base_classifier, n_estimators=10, random_state=42)

In [ ]:
bagging_classifier.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [1]:
y_pred = bagging_classifier.predict(X_test)

NameError: name 'bagging_classifier' is not defined

In [ ]:
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
print("Accuracy:", accuracy)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import ElasticNet, Lasso,LinearRegression,RidgeCV
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns

kf = KFold(n_splits=5,random_state=48,shuffle=True)
n=0

In [ ]:
model2 = ElasticNet(alpha=0.00001)
    model2.fit(X_tr,y_tr)
    pred2[test_idx] = model2.predict(X_val)
    test2 += model2.predict(test[columns])/kf.n_splits
    rmse2 = mean_squared_error(y_val, model2.predict(X_val), squared=False)
    print(": model2 rmse = {}".format(rmse2))

In [ ]:
model3 = xgb.XGBRegressor(**params_xgb)
    model3.fit(X_tr,y_tr,eval_set=[(X_val,y_val)],early_stopping_rounds=200,verbose=False)
    pred3[test_idx] = model3.predict(X_val)
    test3 += model3.predict(test[columns])/kf.n_splits
    rmse3 = mean_squared_error(y_val, model3.predict(X_val), squared=False)
    print(": model4 rmse = {}".format(rmse4))